 CSV que combina Metadata con Finding Annotations. Si hay una fila repetida en Finding Annotations, duplica una en Metadata para no perder esa información.

In [1]:
import pandas as pd

# Ruta a los archivos CSV
finding_annotations_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/original/Vindrmammo/finding_annotations.csv'
metadata_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/original/Vindrmammo/metadata.csv'

# Cargar los CSV en dataframes de pandas
finding_annotations = pd.read_csv(finding_annotations_path)
metadata = pd.read_csv(metadata_path)

# Hacer la unión de los dataframes usando 'image_id' en finding_annotations y 'SOP Instance UID' en metadata
# Primero vamos a asegurarnos que las columnas estén correctamente nombradas y con el tipo de datos correcto

# Verificar si hay columnas extra que podamos eliminar para hacer más limpia la unión
# La columna 'image_id' en finding_annotations tiene que coincidir con 'SOP Instance UID' en metadata
merged_df = pd.merge(finding_annotations, metadata, how='outer', left_on='image_id', right_on='SOP Instance UID')

# Verificar si hay duplicados en 'image_id' y manejarlos creando filas duplicadas de 'SOP Instance UID'
# Duplicamos las filas de image_id repetidos y mantenemos toda la información
duplicated_image_ids = merged_df[merged_df.duplicated(subset='image_id', keep=False)]

# Imprimir cuántos duplicados se encontraron
print(f"Se encontraron {len(duplicated_image_ids)} filas con image_id duplicados")

# Guardar el dataframe combinado en un nuevo CSV
output_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'
merged_df.to_csv(output_csv_path, index=False)

print(f"Archivo combinado guardado en: {output_csv_path}")


Se encontraron 824 filas con image_id duplicados
Archivo combinado guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv


Ahora creamos otro CSV que contenga solo las categorías "Mass" o "Suscipious Calcifications" en la columna finding annotations.
No 2 categorías o más , ni mass y suspicious calcifications juntas. Esto es porque queremos que el sistema reconozca si es una massa o calcificacion benigna, sospechosa o maligna y al agregarle más hallazgos a una imagen esto la condiciona o aumenta las probabilidades de que sea maligna.

In [2]:
import pandas as pd

# Ruta al archivo CSV combinado
combined_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'

# Cargar el CSV combinado
combined_df = pd.read_csv(combined_csv_path)

# Filtrar las filas que tengan exactamente una categoría en 'finding_categories' que sea "Mass" o "Suspicious Calcification"
def filter_mass_or_calcification(categories):
    # Verificar si es una lista de una sola categoría y si esa categoría es "Mass" o "Suspicious Calcification"
    categories_list = eval(categories) if isinstance(categories, str) else categories
    return len(categories_list) == 1 and categories_list[0] in ['Mass', 'Suspicious Calcification']

filtered_df = combined_df[combined_df['finding_categories'].apply(filter_mass_or_calcification)]

# Guardar el nuevo CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

print(f"CSV filtrado guardado en: {filtered_csv_path}")


CSV filtrado guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv


Ahora creamos un CSV con nueva columna image_name para diferenciar las imagenes que tienen varios image_id

In [3]:
import pandas as pd

# Ruta al CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv'

# Cargar el CSV filtrado
filtered_csv = pd.read_csv(filtered_csv_path)

# Crear una nueva columna 'image_name' que sea una copia inicial del 'SOP Instance UID'
filtered_csv['image_name'] = ''

# Agrupar por 'SOP Instance UID' y contar las repeticiones
image_counts = filtered_csv.groupby('SOP Instance UID').cumcount()

# Asignar el nuevo valor en 'image_name' con el formato {SOP Instance UID}_{n}
for index, row in filtered_csv.iterrows():
    sop_uid = row['SOP Instance UID']
    count = image_counts[index]
    filtered_csv.at[index, 'image_name'] = f"{sop_uid}_{count}"

# Guardar el nuevo CSV con la columna 'image_name'
new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv'
filtered_csv.to_csv(new_csv_path, index=False)

print(f"Nuevo CSV guardado en: {new_csv_path}")


Nuevo CSV guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv


Ahora ya tenemos el csv que necesitamos.